# Flipkart EDA assignment

## TODO: Upload csv file before moving to next

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Flipkart Data Engineering").getOrCreate()
file_path = '/FileStore/tables/Flipkart.csv'
flipkart_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [0]:
flipkart_df.printSchema()
flipkart_df.count()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- maincateg: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- actprice1: integer (nullable = true)
 |-- norating1: integer (nullable = true)
 |-- noreviews1: integer (nullable = true)
 |-- star_5f: integer (nullable = true)
 |-- star_4f: integer (nullable = true)
 |-- star_3f: integer (nullable = true)
 |-- star_2f: integer (nullable = true)
 |-- star_1f: integer (nullable = true)
 |-- fulfilled1: integer (nullable = true)

Out[34]: 5244

In [0]:
flipkart_df.show(5)

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|      999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|         1|
|20532|Women Black Flats...|   3.9|    Women|Flipkart|      499|     3015|       404|   1458|    657|    397|    182|    321|         1|
|10648|Women Gold Wedges...|   3.9|    Women|Flipkart|      999|      449|        52|    229|     70|     71|     33|     46|         1|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart|     2999|      290|        40|    141|     51|     49|     17|     32|         1|
|12593|Loafers For Men  ...|   3.9|      

In [0]:
# Display all category names

category_names=flipkart_df.select("maincateg").distinct().show()

+---------+
|maincateg|
+---------+
|     null|
|      Men|
|    Women|
+---------+



In [0]:
# Filter products with rating > 4 and more than 100 reviews
from pyspark.sql.functions import col
product_rate=flipkart_df.select("title").filter((col("Rating")>4.0) & (col('noreviews1')>100)).show()

+--------------------+
|               title|
+--------------------+
|Women Black Wedge...|
|Denill Ankle Leng...|
|     SFG-23 Slippers|
|White casual, gym...|
|Synthetic Leather...|
|Running Shoes For...|
|SM-484 Casuals Fo...|
|Women Black Heels...|
|Women Brown Wedge...|
|GO WALK 5-SPARROW...|
|crocs Unisex's Ra...|
|Men Grey Clogs Sa...|
|Men Navy Sports S...|
|Women Black Wedge...|
|Sneakers For Men ...|
|Men Black Clogs S...|
|Walking Shoes For...|
|Mojaris For Men  ...|
|Women Pink Wedges...|
|              Slides|
+--------------------+
only showing top 20 rows



In [0]:
# Display Products in 'Men' category that are fulfilled
display_products=flipkart_df.select("title").filter(col("maincateg")=="Men").show()


+--------------------+
|               title|
+--------------------+
|Casuals For Men  ...|
|Men's Height Incr...|
|Loafers For Men  ...|
|Canvas Shoes For ...|
|Combo Pack Of 2 C...|
|MOCCASSIN Slip On...|
|Long OfficeWear C...|
|     SFG-23 Slippers|
|Dark Brown Leathe...|
|White casual, gym...|
|Long OfficeWear C...|
|Monk Strap For Me...|
|Running Shoes For...|
|    Men Brown Sandal|
|SM-484 Casuals Fo...|
|Men Hawaii Flip F...|
|Men Blue Clogs Sa...|
|Java 2 IDP Flip F...|
|Derby For Men  (B...|
|Combo pack of 2 s...|
+--------------------+
only showing top 20 rows



In [0]:
# Dsiplay number of products per category

category_counts = flipkart_df.groupBy("maincateg").count().show()

+---------+-----+
|maincateg|count|
+---------+-----+
|     null|   67|
|      Men| 2145|
|    Women| 3032|
+---------+-----+



In [0]:
# Display Average rating per category

from pyspark.sql.functions import avg
avg_rating_per_category = flipkart_df.groupBy("maincateg").agg(avg("Rating").alias("average_rating")).show()

+---------+------------------+
|maincateg|    average_rating|
+---------+------------------+
|     null|             4.225|
|      Men|3.9662970106075246|
|    Women|4.0416582064297915|
+---------+------------------+



In [0]:
# Dsiplay Category with highest average number of reviews

from pyspark.sql.functions import avg
avg_reviews = flipkart_df.groupBy("maincateg") .agg(avg("noreviews1").alias("average_reviews"))
avg_reviews.orderBy("average_reviews", ascending=False).show(1)

+---------+-----------------+
|maincateg|  average_reviews|
+---------+-----------------+
|      Men|759.6554778554779|
+---------+-----------------+
only showing top 1 row



In [0]:
# Top 5 products with highest price. display product name and price
top_expensive_products = flipkart_df.select("title", "actprice1").orderBy("actprice1", ascending=False).limit(5)

top_expensive_products.show(truncate=False)

+----------------------------------------------------------+---------+
|title                                                     |actprice1|
+----------------------------------------------------------+---------+
|GEL-QUANTUM 360 KNIT Running Shoes For Women  (Blue, Grey)|15999    |
|Women Orange Sports Sandal                                |12999    |
|GT-2000 5 (D) Running Shoes For Women  (Blue)             |10999    |
|Hybrid Astro Running Shoes For Men  (Black)               |10999    |
|DEERUPT RUNNER W Sneakers For Women  (White)              |9999     |
+----------------------------------------------------------+---------+



In [0]:
# Display Min, max, and avg price per category

from pyspark.sql.functions import min, max, avg
price_stats = flipkart_df.groupBy("maincateg").agg(
    min("actprice1").alias("min_price"),
    max("actprice1").alias("max_price"),
    avg("actprice1").alias("avg_price")
)
price_stats.show()



+---------+---------+---------+------------------+
|maincateg|min_price|max_price|         avg_price|
+---------+---------+---------+------------------+
|     null|      499|     5999|1574.7313432835822|
|      Men|      139|    10999|  1455.86013986014|
|    Women|      162|    15999|1319.7081134564644|
+---------+---------+---------+------------------+



In [0]:
# Display number of nulls in each column
from pyspark.sql.functions import col, sum
null_counts = flipkart_df.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in flipkart_df.columns
])
null_counts.show()




+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| id|title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|  0|    0|   203|       67|       0|        0|        0|         0|     68|      0|      0|      0|    186|         0|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+



In [0]:
# calculate and display the category name, number of fulfilled, and unfulfilled products
from pyspark.sql.functions import count, when

flipkart_df.groupBy("maincateg").agg(
    count(when(col("fulfilled1") == 1, True)).alias("fulfilled_count"),
    count(when(col("fulfilled1") == 0, True)).alias("unfulfilled_count")
).show()

+---------+---------------+-----------------+
|maincateg|fulfilled_count|unfulfilled_count|
+---------+---------------+-----------------+
|     null|             40|               27|
|      Men|           1352|              793|
|    Women|           1778|             1254|
+---------+---------------+-----------------+



In [0]:
# Display Count of products per category

flipkart_df.groupBy("maincateg").count().show()

+---------+-----+
|maincateg|count|
+---------+-----+
|     null|   67|
|      Men| 2145|
|    Women| 3032|
+---------+-----+



In [0]:
# Display Average rating per category

from pyspark.sql.functions import avg

flipkart_df.groupBy("maincateg").agg(avg("Rating").alias("avg_rating")).show()

+---------+------------------+
|maincateg|        avg_rating|
+---------+------------------+
|     null|             4.225|
|      Men|3.9662970106075246|
|    Women|4.0416582064297915|
+---------+------------------+



In [0]:
# Display Category with highest average number of reviews

from pyspark.sql.functions import avg

flipkart_df.groupBy("maincateg").agg(avg("noreviews1").alias("avg_reviews")).orderBy("avg_reviews", ascending=False).show(1)

+---------+-----------------+
|maincateg|      avg_reviews|
+---------+-----------------+
|      Men|759.6554778554779|
+---------+-----------------+
only showing top 1 row



In [0]:
# Display Bar chart of product count per category

# Use Databricks UI to visualize this as a bar chart
category_counts = flipkart_df.groupBy("maincateg").count()
category_counts.display()


maincateg,count
null,67
Men,2145
Women,3032


Databricks visualization. Run in Databricks to view.

In [0]:
# Bar chart of average rating per category

from pyspark.sql.functions import avg

category_avg_ratings = flipkart_df.groupBy("maincateg").agg(avg("Rating").alias("avg_rating"))
category_avg_ratings.display()

maincateg,avg_rating
null,4.225
Men,3.9662970106075246
Women,4.0416582064297915


Databricks visualization. Run in Databricks to view.

In [0]:
# Display Bar chart of total number of reviews per category
from pyspark.sql.functions import sum

category_total_reviews = flipkart_df.groupBy("maincateg").agg(sum("noreviews1").alias("total_reviews"))
category_total_reviews.display()


maincateg,total_reviews
null,14084
Men,1629461
Women,535290


In [0]:
# Display product name and 5 star rating for those products which have highest 5 star rating

flipkart_df.filter(col("star_5f") == flipkart_df.agg({"star_5f": "max"}).collect()[0][0]).select("title", "star_5f").show()



+--------------------+-------+
|               title|star_5f|
+--------------------+-------+
|Jio-13 Running Sh...| 151193|
|Running Shoes For...| 151193|
+--------------------+-------+

